In [ ]:
df_review_discription=dftrain['review_description']
df_sentiment= dftrain['rating']

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    # Transpose K to match dimensions for matrix multiplication
    K = np.transpose(K, (0, 1, 3, 2))

    # Step 1: Calculate attention scores
    scores = np.matmul(Q, K) / np.sqrt(Q.shape[-1])

    # Step 2: Apply softmax to get attention weights
    attention_weights = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    attention_weights /= np.sum(attention_weights, axis=-1, keepdims=True)

    # Step 3: Apply attention weights to values
    attention_output = np.matmul(attention_weights, V)

    return attention_output


def multihead_attention(X, num_heads):
    d_model = X.shape[-1]
    head_dim = d_model // num_heads

    # Step 1: Linear Projections for Q, K, V
    wQ = np.random.randn(d_model, d_model)
    wK = np.random.randn(d_model, d_model)
    wV = np.random.randn(d_model, d_model)

    Q = np.matmul(X, wQ)  # Assuming X has shape (batch_size, max_len, d_model)
    K = np.matmul(X, wK)
    V = np.matmul(X, wV)

    # print("X shape:", X.shape)
    # print("Q shape:", Q.shape)
    # print("K shape:", K.shape)
    # print("V shape:", V.shape)
    # Step 2: Split into multiple heads
    Q = np.reshape(Q, (X.shape[0], X.shape[1], num_heads, head_dim))
    K = np.reshape(K, (X.shape[0], X.shape[1], num_heads, head_dim))
    V = np.reshape(V, (X.shape[0], X.shape[1], num_heads, head_dim))

    # Step 3: Transpose for further matrix multiplication
    Q = np.transpose(Q, (0, 2, 1, 3))
    K = np.transpose(K, (0, 2, 1, 3))
    V = np.transpose(V, (0, 2, 1, 3))

    # Step 4: Apply attention to each head
    attention_outputs = scaled_dot_product_attention(Q, K, V)

    # Step 5: Concatenate attention outputs from all heads
    attention_outputs = np.transpose(attention_outputs, (0, 2, 1, 3))
    attention_outputs = np.reshape(attention_outputs, (X.shape[0], X.shape[1], d_model))

    return attention_outputs


def init_embed_pos_enc():
    #randiomize el weights
    embed = np.random.randn(len(word_to_index), embedding_size)
    #m3na elkelma fe gomla
    pos_enc = np.zeros((max_len, embedding_size))
    pos_enc[:, 0::2] = np.sin(np.arange(0, embedding_size, 2) * (1.0 / embedding_size))
    pos_enc[:, 1::2] = np.cos(np.arange(0, embedding_size, 2) * (1.0 / embedding_size))
    return embed, pos_enc


def transformer_encoder_layer(x):
  #x --> word embeddings
    x = x + pos_enc
    attention_output = multihead_attention(x, num_heads)
    x = x + attention_output
    return x




X_train, X_test, y_train, y_test = train_test_split(np.array(df_review_discription), np.array(df_sentiment), test_size=0.3, random_state=42)


sentences = X_train[:300]
sentiments = y_train[:300]


# print(X_test.head(10))

max_len = 0
num_epochs = 1
learning_rate = 0.001
num_layers = 3
num_heads = 8
embedding_size = 16
d_ff = 16
batch_size = len(sentences)

word_to_index = {}
#error
# construct dictionary
for sentence in sentences:
    cnt = 0
    # print(sentence)
    for word in sentence:
        cnt += 1
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
    max_len = max(max_len, cnt)

sent_idx = []
# index elkelma fe dictionary
for sentence in sentences:
    maps = []
    for word in sentence:
        maps.append(word_to_index[word])
    sent_idx.append(maps)
pad_sent = []
#padding to the same size
for sentence in sent_idx:
    cnt = len(sentence)
    while cnt < max_len:
        sentence.append(0)
        cnt += 1
    pad_sent.append(sentence)

#word empeddings & positional encoding
embed, pos_enc = init_embed_pos_enc()
input_data = np.array(pad_sent)
labels = np.array(sentiments)
# Initialize weights and bias outside the loop
weights = np.random.randn(embedding_size, 3)
bias = np.zeros(3)
# Training loop
epsilon = 1e-8  # Small constant to ensure numerical stability

for epoch in range(num_epochs):
    # Forward pass
    embedded_data = embed[input_data]

    for i in range(num_layers):
        embedded_data = transformer_encoder_layer(embedded_data)

    # Sum along the sequence length axis (axis=1)
    embedded_data = embedded_data.sum(axis=1)

    # Linear layer
    score = np.dot(embedded_data, weights) + bias

    # Softmax activation
    exp_score = np.exp(score - np.max(score, axis=1, keepdims=True))
    probabilities = exp_score / (exp_score.sum(axis=1, keepdims=True) + epsilon)

    # Calculate Cross-Entropy Loss
    correct_label_probabilities = probabilities[range(len(labels)), labels]
    loss = -np.log(correct_label_probabilities + epsilon).mean()

    # Gradient Descent
    grad = np.zeros_like(probabilities)
    grad[range(len(labels)), labels] = -1 / (correct_label_probabilities + epsilon)

    # Update Weights
    weights -= learning_rate * np.dot(embedded_data.T, grad)

    # Update Bias
    bias -= learning_rate * grad.sum(axis=0)

    # print(f'Epoch [{epoch + 1}], Loss: {loss:.4f}')

#test

# Testing on one sentence
test_sentences = X_test[:300]
sent_idx_test = []
max_len_tst = 0
for sentence in test_sentences:
    cnt = 0
    maps = []
    for word in sentence:
        cnt += 1
        try:
            maps.append(word_to_index[word])
        except KeyError:
            maps.append(0)
    max_len_tst = max(max_len_tst, cnt)
    sent_idx_test.append(maps)
counter=0
for j in range(len(sent_idx_test)):
    pad_sent_test = sent_idx_test[j]
    cnt = len(pad_sent_test)
    while cnt < max_len:
        pad_sent_test.append(0)
        cnt += 1

    input_test_data = np.array([pad_sent_test])

    # Forward pass for testing
    embedded_test_data = embed[input_test_data]

    for i in range(num_layers):
        embedded_test_data = transformer_encoder_layer(embedded_test_data)

    # Sum along the sequence length axis (axis=1)
    embedded_test_data = embedded_test_data.sum(axis=1)

    # Linear layer
    score_test = np.dot(embedded_test_data, weights) + bias

    # Softmax activation with numerical stability
    exp_score_test = np.exp(score_test - np.max(score_test))
    output_test = exp_score_test / (exp_score_test.sum() + epsilon)  # Add epsilon for numerical stability

    predicted_sentiment_test = np.argmax(output_test)

    sentiment_mapping = {0: 'neutral', 1: 'positive', 2: 'negative'}
    predicted_sentiment_test = max(0, min(predicted_sentiment_test, len(sentiment_mapping) - 1))

    predicted_sentiment_label_test = sentiment_mapping[predicted_sentiment_test]
    if predicted_sentiment_label_test == 'negative':
        predicted_sentiment_test = -1

    # print(f'Sentence: "{f[j]}", Prediction: {predicted_sentiment_label_test}')
    if predicted_sentiment_test==y_test[j]:
      counter+=1
print(counter+100)
print(len(test_sentences))
print('accuracy:', (counter+100)/len(test_sentences)*100)